## Analizador Lexico
Un analizador lexico transforma una entrada en tokens

In [23]:
from enum import Enum

class TipoToken(Enum):  #sistema para clasificación de palabras y conectores
    O = 1
    Y = 2
    S = 3
    N = 4
    E = 5
    P = 6

In [24]:
def Analizador_lexico(entrada : str) -> list:
    lexemas.clear()
    tokens = entrada.lower().split()
    for token in tokens:
        if token == 'o':
            lexemas.append({"valor": token , "tipo": TipoToken.O })
        elif token == 'y':
            lexemas.append({"valor": token , "tipo": TipoToken.Y})
        elif token == 'si':
            lexemas.append({"valor": token , "tipo": TipoToken.S})
        elif token == 'entonces':
            lexemas.append({"valor": token , "tipo": TipoToken.E})
        elif token == 'no':
            lexemas.append({"valor": token , "tipo": TipoToken.N})
        else :
            lexemas.append({"valor":token , "tipo": TipoToken.P })
            
    return(lexemas)

## Analizador Sintactico

In [25]:
class EstadosAnalizador(Enum):  #Sistema de guardado de Estados
    INICIO = 1
    PRECEDENTE = 2
    NEGACION = 3
    CONSECUENTE = 4
    DISYUNCION = 5
    CONJUNCION = 6
    ATOMO = 7
    ERROR1 = 8  #se esperaba un 'si', 'no' o un atomo
    ERROR2 = 9  #se esperaba un 'no' o un atomo
    ERROR3 = 10 #se esperaba un 'y', 'o', 'entonces' o un atomo
    ERROR4 = 11 #se esperaba un atomo
    ERROR5 = 12 #no se encontro un 'si' antes de 'entonces'
    FIN = 13

In [32]:
def Analizador_sintactico(lexemas : list):
    regla = ""
    pila = ""
    prec = 0
    pila = []
    prop = ""

    EstadoActual = EstadosAnalizador.INICIO

    for lexema in lexemas:
        if EstadoActual == EstadosAnalizador.INICIO:
            if lexema['valor'] == "si":
                EstadoActual = EstadosAnalizador.PRECEDENTE
                prec = 1
            elif lexema['valor'] == "no":
                EstadoActual = EstadosAnalizador.NEGACION
                if regla:
                    regla += " "
                regla += "~"
            elif lexema['tipo'] == TipoToken.P:
                EstadoActual = EstadosAnalizador.ATOMO
                pila.append(lexema['valor'])
            else:
                EstadoActual = EstadosAnalizador.ERROR1

        elif EstadoActual == EstadosAnalizador.PRECEDENTE:
            if lexema['valor'] == "no":
                EstadoActual = EstadosAnalizador.NEGACION
                if regla:
                    regla += " "
                regla += "~"
            elif lexema['tipo'] == TipoToken.P:
                EstadoActual = EstadosAnalizador.ATOMO
                pila.append(lexema['valor'])
            else:
                EstadoActual = EstadosAnalizador.ERROR2

        elif EstadoActual == EstadosAnalizador.NEGACION:
            if lexema['tipo'] == TipoToken.P:
                EstadoActual = EstadosAnalizador.ATOMO
                pila.append(lexema['valor'])
            else:
                EstadoActual = EstadosAnalizador.ERROR3

        elif EstadoActual == EstadosAnalizador.CONSECUENTE:
            if lexema['tipo'] == TipoToken.P:
                EstadoActual = EstadosAnalizador.ATOMO
                pila.append(lexema['valor'])
            elif lexema['valor'] == "no":
                EstadoActual = EstadosAnalizador.NEGACION
                if regla:
                    regla += " "
                regla += "~"
            else:
                EstadoActual = EstadosAnalizador.ERROR2

        elif EstadoActual == EstadosAnalizador.ATOMO:
            if lexema['valor'] == "entonces" and prec == 1:
                EstadoActual = EstadosAnalizador.CONSECUENTE
                prop = " ".join(pila)
                if regla:
                    regla += " "
                regla += prop
                if regla:
                    regla += " "
                regla += "->"
                pila.clear()
                prop = ""
            elif lexema['valor'] == "y":
                EstadoActual = EstadosAnalizador.CONJUNCION
                prop = " ".join(pila)
                if regla:
                    regla += " "
                regla += prop
                if regla:
                    regla += " "
                regla += "&"
                pila.clear()
                prop = ""
            elif lexema['valor'] == "o":
                EstadoActual = EstadosAnalizador.DISYUNCION
                prop = " ".join(pila)
                if regla:
                    regla += " "
                regla += prop
                if regla:
                    regla += " "
                regla += "V"
                pila.clear()
                prop = ""
            elif lexema['tipo'] == TipoToken.P:
                EstadoActual = EstadosAnalizador.ATOMO
                pila.append(lexema['valor'])
            elif lexema['valor'] == "entonces" and prec == 0:
                EstadoActual = EstadosAnalizador.ERROR5
            else:
                EstadoActual = EstadosAnalizador.ERROR3

        elif EstadoActual == EstadosAnalizador.CONJUNCION:
            if lexema['tipo'] == TipoToken.P:
                EstadoActual = EstadosAnalizador.ATOMO
                pila.append(lexema['valor'])
            elif lexema['valor'] == "no":
                EstadoActual = EstadosAnalizador.NEGACION
                if regla:
                    regla += " "
                regla += "~"
            else:
                EstadoActual = EstadosAnalizador.ERROR2

        elif EstadoActual == EstadosAnalizador.DISYUNCION:
            if lexema['tipo'] == TipoToken.P:
                EstadoActual = EstadosAnalizador.ATOMO
                pila.append(lexema['valor'])
            elif lexema['valor'] == "no":
                EstadoActual == EstadosAnalizador.NEGACION
                if regla:
                    regla += " "
                regla += "~"
            else:
                EstadoActual = EstadosAnalizador.ERROR2

        elif EstadoActual == EstadosAnalizador.ERROR1:
            print("se esperaba un 'si', 'no' o un atomo")
            regla = ""
            prop = ""
            pila.clear()
            break
        elif EstadoActual == EstadosAnalizador.ERROR2:
            print("se esperaba un 'no' o un atomo")
            regla = ""
            prop = ""
            pila.clear()
            break
        elif EstadoActual == EstadosAnalizador.ERROR3:
            regla = ""
            prop = ""
            pila.clear()
            print("se esperaba un 'y', 'o', 'entonces' o un atomo")
            break
        elif EstadoActual == EstadosAnalizador.ERROR4:
            print("se esperaba un atomo")
            regla = ""
            prop = ""
            pila.clear()
            break
        elif EstadoActual == EstadosAnalizador.ERROR5:
            print("no se encontro un 'si' antes de 'entonces'")
            regla = ""
            prop = ""
            pila.clear()
            break
        else:
            EstadoActual == EstadosAnalizador.FIN

    if EstadoActual != (EstadosAnalizador.ERROR1 and EstadosAnalizador.ERROR2 and EstadosAnalizador.ERROR3 and EstadosAnalizador.ERROR4 and EstadosAnalizador.ERROR5):
        prop = " ".join(pila)
        if regla and prop:
            regla += " " + prop
        else:
            regla = prop
        l_reglas.append(regla)
        prop = ""
        pila.clear()
        regla = ""
    
    return(l_reglas)

In [33]:
lexemas = []
l_reglas = []
Palabras = []

frases = [
    "Si es de dia o es de noche entonces existo",
    "Es de dia"]

for frase in frases:
    print(frase)
    Palabras.clear()
    Palabras = Analizador_lexico(frase)
    Analizador_sintactico(Palabras)

l_reglas

Si es de dia o es de noche entonces existo
Es de dia


['es de dia V es de noche -> existo', 'es de dia']

## Prueba de lectura de archivos

In [34]:
lexemas = []
l_reglas = []
Palabras = []

with open("reglas-texto.txt", "r") as archivo:
    lineas = archivo.readlines()

erease = "\n"
enun = []
for linea in lineas:
    ent = linea.replace(erease, "")
    enun.append(ent)

for frase in enun:
    print(frase)
    Palabras.clear()
    Palabras = Analizador_lexico(frase)
    Analizador_sintactico(Palabras)

l_reglas

Si no tiene orejas puntiagudas entonces no es un elfo
Si tiene pelaje o sus ojos parecen animales entonces no es un elfo
Si no es un elfo alto entonces es un elfo oscuro o es un elfo del bosque
Si tiene piel oscura entonces es un elfo o es un guarda rojo
Si tiene piel clara y no tiene orejas puntiagudas entonces es un nordico
Si tiene colmillos entonces es un orco o es un kahjiit
Si es un ser humano y no es nordico o no es un guarda rojo entonces es un imperial o es un breton
Si usa magia de forma natural entonces es un breton o es un elfo oscuro o es un alto elfo
Si es muy habil con las armas pesadas entonces es un orco o es un nordico o es un guarda rojo
Si es muy habil al ser sigiloso entonces es un kahjiit o un elfo del bosque
Es un kahjiit entonces puede ver en la oscuridad
no se encontro un 'si' antes de 'entonces'


['~ tiene orejas puntiagudas -> ~ es un elfo',
 'tiene pelaje V sus ojos parecen animales -> ~ es un elfo',
 '~ es un elfo alto -> es un elfo oscuro V es un elfo del bosque',
 'tiene piel oscura -> es un elfo V es un guarda rojo',
 'tiene piel clara & ~ tiene orejas puntiagudas -> es un nordico',
 'tiene colmillos -> es un orco V es un kahjiit',
 'es un ser humano & ~ es nordico V ~ es un guarda rojo -> es un imperial V es un breton',
 'usa magia de forma natural -> es un breton V es un elfo oscuro V es un alto elfo',
 'es muy habil con las armas pesadas -> es un orco V es un nordico V es un guarda rojo',
 'es muy habil al ser sigiloso -> es un kahjiit V un elfo del bosque']

## Analizador semántico